
## Optimization notebook for ETR's Draftkings golf projections.  This notebook uses Google ortools and their linear solver.  

**NOTE: This notebook is for example purposes only and has no guarantees for accuracy.  Use at your own risk.** 


### Contents
1. [Overview](#overview)
2. [Imports](#imports)
3. [Data Loading and Cleaning](#data_loading)
4. [Solve For DK Points and Ceiling Optimal Rosters](#solver)

<a id='overview'></a>
## 1. Overview

This notebook solves the n-dimensional 0-1 knapsack problem of roster selection for DFS golf.  A roster of 6 players with a maximum cumulative salary of $50000 is allowed.  This notebook uses Establish The Run's DK Points projections to find multiple rosters.  The projections are available for download through ETR's golf subscription.

The following is the constraint problem setup:

maximize $\sum_{i=1}^n v_i x_i$

subject to 

$\sum_{i=1}^n w_i x_i <= 50000$, 

$\sum_{i=1}^n x_i == 6$,

$x_i \in \{0, 1\}$

Here, $x_i$ is a 0-1 variable representing whether player i is included in the roster.  $v_i$ represents ETR's points projection for player i.  $w_i$ represents the DK salary for player i.


<a id='imports'></a>
## 2. Imports
#### This python notebook uses the pandas and ortools packages.

In [1]:
import numpy as np
import pandas as pd
from ortools.linear_solver import pywraplp

<a id='data_loading'></a>
## 3. Data Loading and Cleaning
#### Load ETR's csv download into a pandas dataframe.  Change the folder location if the csv is not in the same folder.  Change the title of the csv to match the current weeks projections.

In [2]:
current_folder = "./"
file_path = current_folder + 'ETR DraftKings Golf Projections5_11_22.csv'
df= pd.read_csv(file_path)
df.head()

,Golfer,Rd 1 Tee Time,Rd 2 Tee Time,DK Salary,DK Points,DK Value,DK Ownership,DK Ceiling,Make Cut Odds,Volatility,Slate Name,Slate ID
0,Scottie Scheffler,13:06,7:55,10900.0,93.8,8.0,25.6%,134.7,80.7%,1.7,Scottie Scheffler,22529105.0
1,Justin Thomas,12:55,7:44,10600.0,91.1,6.1,24.2%,132.1,79.3%,2.1,Justin Thomas,22529106.0
2,Xander Schauffele,7:55,13:06,9700.0,85.0,5.6,18.4%,126.8,75.2%,3.3,Xander Schauffele,22529110.0
3,Sam Burns,12:55,7:44,9500.0,84.0,5.8,22.6%,127.6,72.9%,4.0,Sam Burns,22529111.0
4,Dustin Johnson,7:55,13:06,10200.0,82.0,3.1,11.3%,126.3,70.8%,4.7,Dustin Johnson,22529107.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155 entries, 0 to 154
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Golfer         155 non-null    object 
 1   Rd 1 Tee Time  155 non-null    object 
 2   Rd 2 Tee Time  155 non-null    object 
 3   DK Salary      152 non-null    float64
 4   DK Points      155 non-null    float64
 5   DK Value       152 non-null    float64
 6   DK Ownership   155 non-null    object 
 7   DK Ceiling     155 non-null    float64
 8   Make Cut Odds  155 non-null    object 
 9   Volatility     155 non-null    float64
 10  Slate Name     155 non-null    object 
 11  Slate ID       152 non-null    float64
dtypes: float64(6), object(6)
memory usage: 14.7+ KB


In [4]:
df['projected_ownership'] = df['DK Ownership'].str.rstrip('%').astype('float') 
# Put in a minimum ownership of 0.1% for us in product ownership calculation
df.loc[df['projected_ownership'] < 0.1, 'projected_ownership'] = 0.1
df['cut_odds'] = df['Make Cut Odds'].str.rstrip('%').astype('float')
df.describe()

,DK Salary,DK Points,DK Value,DK Ceiling,Volatility,Slate ID,projected_ownership,cut_odds
count,152.000000,155.000000,152.000000,155.000000,155.000000,1.520000e+02,155.000000,155.000000
mean,6987.500000,58.878065,-0.673684,98.348387,7.749677,2.252983e+07,4.013548,46.615484
std,990.772827,10.922467,2.081032,13.441419,1.646459,3.516633e+03,5.756995,13.626074
min,6000.000000,25.800000,-2.000000,33.500000,1.000000,2.252910e+07,0.100000,2.200000
25%,6300.000000,52.800000,-2.000000,92.800000,7.000000,2.252914e+07,0.300000,39.250000
50%,6700.000000,58.100000,-1.800000,98.300000,8.300000,2.252918e+07,1.200000,46.500000
75%,7300.000000,65.350000,0.000000,105.050000,9.000000,2.252922e+07,5.300000,55.950000
max,10900.000000,93.800000,8.000000,134.700000,10.000000,2.254883e+07,25.600000,80.700000


#### Some minimal data cleanup.  May or may not delete players in the field depending on the csv accuracy.

In [5]:
print("Players being deleted due to lack of some data. Manually fix data in CSV if this is not correct. \n\n", 
      df[df.isna().any(axis=1)])
df = df.dropna()

Players being deleted due to lack of some data. Manually fix data in CSV if this is not correct. 

               Golfer Rd 1 Tee Time Rd 2 Tee Time  DK Salary  DK Points  \
105   Chase Seiffert         13:28          8:18        NaN       54.0   
118  Satoshi Kodaira          7:33         12:43        NaN       52.5   
143        Jim Knous         12:21          7:12        NaN       45.4   

     DK Value DK Ownership  DK Ceiling Make Cut Odds  Volatility  \
105       NaN         0.5%        94.6         41.3%         9.2   
118       NaN         0.2%        92.9         39.1%         9.4   
143       NaN         0.0%        84.1         28.9%         9.1   

          Slate Name  Slate ID  projected_ownership  cut_odds  
105   Chase Seiffert       NaN                  0.5      41.3  
118  Satoshi Kodaira       NaN                  0.2      39.1  
143        Jim Knous       NaN                  0.1      28.9  


#### Load data into lists for the solver.  Change data to integers and adjust data scale.

In [6]:
# Adjust salary and points for solver use.
salary_divisor = 100
dk_points_multiplier = 10
salaries = df['DK Salary'].values / salary_divisor
salaries = [int(w) for w in salaries]
dk_points = df['DK Points'].values
dk_points = [int(i * dk_points_multiplier) for i in dk_points]
ceiling = df['DK Ceiling'].values
ceiling = [int(i * dk_points_multiplier) for i in ceiling]
names =  df['Golfer'].to_list()
print('lengths', len(names), len(salaries), len(dk_points))

lengths 152 152 152


In [7]:
# Load data in dictionary for the solver.
data = {}
data['ownership'] = df['projected_ownership'].values
data['weights'] = salaries
data['values'] = dk_points 
data['points'] = dk_points
data['ceiling'] = ceiling
data['sigma'] = df['Volatility'].values
data['num_items'] = len(data['weights'])
data['all_items'] = range(data['num_items'])
data['cut_odds'] = df['cut_odds'].values
data['names'] = names
data['max_weight'] = 500
data['max_items'] = 6

<a id='solver'></a>
## 4. Solve For DK Points and Ceiling Optimal Rosters

#### Use the third party solver SCIP, which is included in ortools.  Information on SCIP [here.](https://www.scipopt.org)

### Optimal based on DK Points projection.

In [8]:
solver = pywraplp.Solver.CreateSolver('SCIP')

# 0/1 variable to show which item is included in the roster.
x = {}
for i in data['all_items']:
    x[i] = solver.BoolVar(f'x_{i}')
    
# Constraints
# Constrain to roster of 6 spots.
solver.Add(sum(x[i] for i in data['all_items']) == data['max_items'])
# Constrain to max salary <= 50000
solver.Add(sum([x[i] * data['weights'][i] for i in data['all_items']]) <= data['max_weight'])

# Objective Function
objective = solver.Objective()
for i in data['all_items']:
    objective.SetCoefficient(x[i], data['values'][i])
objective.SetMaximization()

# Solve
solver_parameters = pywraplp.MPSolverParameters()
solver_parameters.SetDoubleParam(pywraplp.MPSolverParameters.PRIMAL_TOLERANCE, 0.0001)
status = solver.Solve(solver_parameters)

# Results of solver.
if status == pywraplp.Solver.OPTIMAL:
    print(f'Objective: {(objective.Value() / dk_points_multiplier):.2f}')
    total_weight = 0
    product_ownership = 1
    estimate_cut_odds = 1
    for i in data['all_items']:
        if x[i].solution_value() > 0:
            player_salary = data['weights'][i] * salary_divisor
            points = data['points'][i] / dk_points_multiplier
            print(
                f"{data['names'][i]:20} salary: {player_salary:5} points: {points:.1f} ownership: {data['ownership'][i]:5} cut odds: {data['cut_odds'][i]}"
            )
            total_weight += player_salary
            product_ownership *= data['ownership'][i]
            estimate_cut_odds *= (data['cut_odds'][i] / 100)

    print(f'Total Salary: {total_weight}')
    print(f'Projected Product Ownership: {product_ownership:.0f}')
    print(f'Probablity Estimate All 6 To Make Cut: {estimate_cut_odds:.4f}')
    print('\n')

Objective: 457.70
Justin Thomas        salary: 10600 points: 91.1 ownership:  24.2 cut odds: 79.3
Xander Schauffele    salary:  9700 points: 85.0 ownership:  18.4 cut odds: 75.2
Aaron Wise           salary:  8100 points: 74.9 ownership:  18.2 cut odds: 64.7
Si Woo Kim           salary:  7800 points: 73.2 ownership:  12.4 cut odds: 63.8
Matthew Nesmith      salary:  7300 points: 69.5 ownership:   9.1 cut odds: 61.1
Adam Schenk          salary:  6500 points: 64.0 ownership:   4.7 cut odds: 54.8
Total Salary: 50000
Projected Product Ownership: 4297991
Probablity Estimate All 6 To Make Cut: 0.0824




### Find Optimal Using Ceiling

In [9]:
ceiling_solver = pywraplp.Solver.CreateSolver('SCIP')
x = {}
for i in data['all_items']:
    x[i] = ceiling_solver.BoolVar(f'x_{i}')

# Constraints
# Constrain to roster of 6 spots.
ceiling_solver.Add(sum(x[i] for i in data['all_items']) == data['max_items'])
# Constrain to max salary <= 50000
ceiling_solver.Add(sum([x[i] * data['weights'][i] for i in data['all_items']]) <= data['max_weight'])

# Objective Function
objective = ceiling_solver.Objective()
for i in data['all_items']:
    objective.SetCoefficient(x[i], data['ceiling'][i])
objective.SetMaximization()

# Solve
solver_parameters = pywraplp.MPSolverParameters()
solver_parameters.SetDoubleParam(pywraplp.MPSolverParameters.PRIMAL_TOLERANCE, 0.0001)
status = ceiling_solver.Solve(solver_parameters)

# Results of solver.
if status == pywraplp.Solver.OPTIMAL:
    print(f'Objective: {(objective.Value() / dk_points_multiplier):.2f}')
    total_weight = 0
    product_ownership = 1
    estimate_cut_odds = 1
    for i in data['all_items']:
        if x[i].solution_value() > 0:
            player_salary = data['weights'][i] * salary_divisor
            points = data['ceiling'][i] / dk_points_multiplier
            print(
                f"{data['names'][i]:20} salary: {player_salary:5} ceiling: {points:.1f} ownership: {data['ownership'][i]:5} cut odds: {data['cut_odds'][i]}"
            )
            total_weight += player_salary
            product_ownership *= data['ownership'][i]
            estimate_cut_odds *= (data['cut_odds'][i] / 100)

    print(f'Total Salary: {total_weight}')
    print(f'Projected Product Ownership: {product_ownership:.0f}')
    print(f'Probablity Estimate All 6 To Make Cut: {estimate_cut_odds:.4f}')
    print('\n')

Objective: 707.20
Xander Schauffele    salary:  9700 ceiling: 126.8 ownership:  18.4 cut odds: 75.2
Sam Burns            salary:  9500 ceiling: 127.6 ownership:  22.6 cut odds: 72.9
Aaron Wise           salary:  8100 ceiling: 117.3 ownership:  18.2 cut odds: 64.7
Si Woo Kim           salary:  7800 ceiling: 114.9 ownership:  12.4 cut odds: 63.8
Seamus Power         salary:  7900 ceiling: 113.9 ownership:  10.1 cut odds: 63.6
Patrick Rodgers      salary:  7000 ceiling: 106.7 ownership:  11.1 cut odds: 56.5
Total Salary: 50000
Projected Product Ownership: 10521162
Probablity Estimate All 6 To Make Cut: 0.0813




### Create multiple lineups using player projections as a base.  
Adjust the base by sampling from the normal distribution using the points projection as a mean and the player volatility as standard deviation.  

In [10]:
def solve_for_optimal(data, data_to_maximize='values'):
    solver = pywraplp.Solver.CreateSolver('SCIP')
    x = {}
    for i in data['all_items']:
        x[i] = solver.BoolVar(f'x_{i}')
    # Constraints
    # Constrain to roster of 6 spots.
    solver.Add(sum(x[i] for i in data['all_items']) == data['max_items'])
    # Constrain to max salary <= 50000
    solver.Add(sum([x[i] * data['weights'][i] for i in data['all_items']]) <= data['max_weight'])
    objective = solver.Objective()
    for i in data['all_items']:
        objective.SetCoefficient(x[i], data[data_to_maximize][i])
    objective.SetMaximization()
    solver_parameters = pywraplp.MPSolverParameters()
    solver_parameters.SetDoubleParam(pywraplp.MPSolverParameters.PRIMAL_TOLERANCE, 0.0001)
    status = solver.Solve(solver_parameters)
    # Results of solver.
    if status == pywraplp.Solver.OPTIMAL:
        print(f'Objective: {(objective.Value() / dk_points_multiplier):.2f}')
        total_weight = 0
        product_ownership = 1
        estimate_cut_odds = 1
        for i in data['all_items']:
            if x[i].solution_value() > 0:
                player_salary = data['weights'][i] * salary_divisor
                points = data[data_to_maximize][i] / dk_points_multiplier
                print(
                    f"{data['names'][i]:20} salary: {player_salary:5} points: {points:.1f} ownership: {data['ownership'][i]:5} cut odds: {data['cut_odds'][i]}"
                )
                total_weight += player_salary
                product_ownership *= data['ownership'][i]
                estimate_cut_odds *= (data['cut_odds'][i] / 100)

        print(f'Total Salary: {total_weight}')
        print(f'Projected Product Ownership: {product_ownership:.0f}')
        print(f'Probablity Estimate All 6 To Make Cut: {estimate_cut_odds:.4f}')
        print('\n')
    else:
        print('The problem does not have an optimal solution.')
    

In [11]:
# Generate lineups.
number_lineups_to_generate = 5
for i in range(number_lineups_to_generate):
    data['values'] = np.random.normal(data['points'], data['sigma'])
    print(f'Roster: {i}')
    solve_for_optimal(data, data_to_maximize='values')

Roster: 0
Objective: 460.06
Scottie Scheffler    salary: 10900 points: 94.0 ownership:  25.6 cut odds: 80.7
Sam Burns            salary:  9500 points: 84.5 ownership:  22.6 cut odds: 72.9
Aaron Wise           salary:  8100 points: 75.1 ownership:  18.2 cut odds: 64.7
Si Woo Kim           salary:  7800 points: 74.0 ownership:  12.4 cut odds: 63.8
Charles Howell III   salary:  7200 points: 68.3 ownership:  12.3 cut odds: 59.4
Harry Higgs          salary:  6500 points: 64.1 ownership:   5.3 cut odds: 53.2
Total Salary: 50000
Projected Product Ownership: 8511821
Probablity Estimate All 6 To Make Cut: 0.0767


Roster: 1
Objective: 456.00
Justin Thomas        salary: 10600 points: 91.1 ownership:  24.2 cut odds: 79.3
Sam Burns            salary:  9500 points: 83.6 ownership:  22.6 cut odds: 72.9
Aaron Wise           salary:  8100 points: 74.6 ownership:  18.2 cut odds: 64.7
Si Woo Kim           salary:  7800 points: 72.7 ownership:  12.4 cut odds: 63.8
Bubba Watson         salary:  7500 poin